In [1]:
# inspired by VPT cleaning process, use model, preprocess = clip.load('RN50x64', device, jit=True)
#!pip install matplotlib
#!pip install -U yt-dlp
#!pip install ffmpeg-python

import yt_dlp
import matplotlib
from yt_dlp import YoutubeDL
import random
import os


def s_to_HHmmss(secs):
    mins = (secs - secs%60) / 60
    secs = secs%60
    hours = (mins - mins%60) / 60
    mins = mins%60
    
    if hours>0:
        HHmmss = ":".join([str(int(hours)),str(int(mins)),str(secs)])
    elif mins>0:
        HHmmss = ":".join([str(int(mins)),str(secs)])
    else:
        HHmmss = str(secs)
            
    return HHmmss

Duplicate key in file PosixPath('/home/idmi/miniconda3/envs/minerl/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 603 ('image.interpolation:\tnone')


In [2]:
# OPEN VIDEO FILE HEADER, READ SEEN LINKS/CHANNELS
header_file = open("spamham_samples/header.txt", "r")
header_text = header_file.readlines()
header_file.close()

all_links = []
all_channels = []

# https://pypi.org/project/yt-dlp/#usage-and-options
def longer_than_a_minute(info, *, incomplete):
    """Download only videos longer than 10 minutes"""
    duration = info.get('duration')
    if duration and duration < 600:
        return 'The video is too short'
YDL_OPTIONS = {'noplaylist':True, 'Download':False, 'match_filter':longer_than_a_minute, 'ignoreerrors':True}

for line in header_text:
    link = line
    if (not (link in all_links)) and (link!=''):
        all_links.append(link)

In [ ]:
search_queries = [
#    "minecraft fresh start new world creative",
#    "minecraft new world modded",
#    "minecraft guide",
#    "minecraft speedrun",
#    "minecraft how to redstone",
#    "minecraft new world tutorial",
#    "minecraft creative guide",
#    "minecraft tutorial how to survival vanilla new world lets play",
#    "minecraft basics",
#    "minecraft for beginners",
#    "minecraft how to hardcore guide",
#    "minecraft how to survival 1.16 guide",

    'minecraft "vanilla" "new world" "survival" "gameplay" -rtx -feature -mod -dragon -lucky -bedrock -pocket -unturned -roblox -fps -shader -server -smp -dimension -"how to" -top',
    'minecraft "vanilla" "new world" "survival"',
    'minecraft "vanilla" "new world" "survival" "lets play"',
    'minecraft "vanilla" "new world" "survival" "lets play" "tutorial"',
    'minecraft "vanilla" "new world" "survival" "gameplay" -rtx -feature -mod -dragon -lucky -bedrock -pocket -unturned -roblox -fps -shader -server -smp -dimension -"how to" -top vod',
    'minecraft longplay',
    ''

#    "minecraft stream",
#    "minecraft lets play",
#    "minecraft gameplay survival",
#    "minecraft gameplay hardcore",
#    "minecraft lets play",
#    "minecraft tutorial",
#    "minecraft new world survival",
#    "minecraft hardcore",
#    "minecraft gameplay surival",
#    "minecraft longplay",
    ]

for query in search_queries:

    search_query = query
    
    #"minecraft tutorial how to hardcore vanilla new world lets play"
    # minecraft vanilla lets play stream
    #"minecraft guide walkthrough survival new world 1.16"
    #"minecraft survival hardcore tutorial how to guide learn basics beginner lets play walkthrough gameplay new world first night first day 1.16"
    #"minecraft new world creative vanilla"
    #"minecraft creative mode how to"
    # "minecraft shaders"
    #"minecraft console edition pocket edition ps3 ps4 ps5 xbox bedrock"
    #"playing minecraft with multiplayer server"
    # minecraft creative 1.16
    # minecraft creative
    # minecraft lets play stream mod
    # minecraft mod technic
    # minecraft mod

    with yt_dlp.YoutubeDL(YDL_OPTIONS) as ydl:
        search_results = ydl.extract_info(f"ytsearchall:{search_query}", download=False)['entries']

    # DOWNLOAD RANDOM FRAMES FROM EACH LINK
    #https://pypi.org/project/yt-dlp/



    # dont re-download frames from videos which have already hadframes downloaded
    with open("spamham_samples/header.txt","r") as used_links:
        used_links = used_links.readlines()
        for i in range(len(used_links)):
            used_links[i] = used_links[i].split('\n')[0]



    # for every link:
    for i in range(len(search_results)):

        
        # separate out video info
        video = search_results[i]
        try:
            video['duration']
        except:
            continue

        if video['duration']>300 and not(video['id'] in used_links):      ############################# PRE-FILTERING FILTERING:

            video_link = video['id']
            video_name = video['title']
            video_channel = video['channel']
            filename = video_link

            print("getting new video!")
            # SELECT RANDOM FRAME
            duration = video['duration']
            random_sec = min(random.randint(0, 10*60), duration-2)
            random_sec_formatted = "*"+random_sec+"-"+random_sec+1


            # DOWNLOAD SHORT VIDEO SECTION
            full_link = str("https://www.youtube.com/watch?v=%s"%video_link)
            print(full_link)
            print(random_sec_formatted)

            filename_ext = filename+'.mp4'
            command_download_frames = str("yt-dlp -S res:720,ext:mp4,fps:24 -c --recode mp4 --download-sections %s %s --output spamham_samples/%s.mp4" %(random_sec_formatted, full_link, filename))
            #command_get_1_frame = str('ffmpeg -i spamham_samples/%s -vcodec png -ss 0 -vframes 1 -an -f rawvideo spamham_samples/frames/%s.png -y' %(filename_ext, filename) )
            command_get_1_frame = str('ffmpeg -i spamham_samples/%s -r 1 -vframes 1 -f image2 -q:v 1 spamham_samples/frames/%s.jpeg -y' %(filename_ext, filename) )
            command_del_video = str('rm spamham_samples/%s.mp4' %filename)
            command_all_in_1 = command_download_frames +' && ' + command_get_1_frame +' && ' + command_del_video
            os.system(command_all_in_1)

            # CROP VIDEO SECTION TO FIRST FRAME
            #"https://www.youtube.com/watch?v={sID}".format(sID=video_link)
            #label = get_response(filename+".jpg")
            line = str(filename+"\n")

            header_file = open("spamham_samples/header.txt","a")
            sources_file = open("spamham_samples/sources.txt","a")
            header_file.write(line)
            sources_file.write(video_channel+","+full_link+"\n")
            header_file.close()
            sources_file.close()

In [25]:
# import
import yt_dlp
import matplotlib
from yt_dlp import YoutubeDL
import random
import os



#yt-dlp --flat-playlist -i --print-to-file url file.txt https://www.youtube.com/playlist?list=PLvgL9fMpgUKpgGsI4EBRAB-1xM-v0rCJ6 command to take aplaylist of videos and get all video IDs. useful to find a palylist of good minecraft videos (often made by a channel owner for organising a series) and gettign all individual videos in there to be used as examples of clean videos




# load clean ids file
clean_ids = open('spamham_samples/manual_clean_ids.txt', 'r')
clean_ids = clean_ids.readlines()

# read already used files
used_links = os.listdir('spamham_samples/frames')
used_links = [filename.split('.')[0] for filename in used_links]

# for file in clean_ids.txt:
for id in clean_ids:
    id = id.split('\n')[0]
    
    if (id in used_links):
       continue

    print('|'+id+'|')
    
    full_link = str("https://www.youtube.com/watch?v=%s"%id)
    
    print('get vid info')
    #YT-DLP get video info
    YDL_OPTIONS = {'noplaylist':True, 'Download':False}
    with yt_dlp.YoutubeDL(YDL_OPTIONS) as ydl:
        video = ydl.extract_info(full_link, download=False)
        video_channel = video['channel']
        duration = video['duration']

    # ONLY GET 1 FRAME PER VIDEO - multiple per video will just overfit - we are already overfitting to the same MC youtuber, multiple frames per vidfeo will hurt generalisation too much
    print('get random frame')
    #@@@ download random second for each hour of gameplay | ffmpeg 1st frame | delete mp4
    #@@@random_second = random.randint((h*60*30)+180, ((h+1)*30*60)-180)
    random_second = random.randint(0, duration-2)
    random_sec_formatted = '*'+str(random_second)+'-'+str(random_second+1)
    print(random_sec_formatted)
    filename = id
    print(filename)

    # save frame under filename: id + .#N.jpeg
    full_link = str("https://www.youtube.com/watch?v=%s"%id)

    ### LINUX
    #command_download_frames = str("yt-dlp -S res:720,ext:mp4,fps:24 -c --recode mp4 --download-sections %s %s --output spamham_samples/%s.mp4" %(random_sec_formatted, full_link, filename))
    #command_get_1_frame = str('ffmpeg -i spamham_samples/%s -r 1 -vframes 1 -f image2 -q:v 1 spamham_samples/frames/%s.jpeg -y' %(filename+'.mp4', filename) )
    #command_del_video = str('rm spamham_samples/%s.mp4' %filename)
    #command_all_in_1 = command_download_frames +' && ' + command_get_1_frame +' && ' + command_del_video
    #print(command_all_in_1)
    #print('running donwload, extract, delete cmd...',end='')
    #print(os.system(command_all_in_1))

    ### WINDOWS
    command_download_frames = str("yt-dlp -S res:720,ext:mp4,fps:24 -c --recode mp4 --download-sections %s %s --output spamham_samples/%s.mp4 --retries infinite --fragment-retries infinite" %(random_sec_formatted, full_link, filename))
    command_get_1_frame = str('ffmpeg -i spamham_samples/'+filename+'.mp4'+' -r 1 -vframes 1 -f image2 -q:v 1 spamham_samples/frames/'+filename+'.%03d.jpeg -y')
    command_del_video = str('del spamham_samples\\%s.mp4' %filename)
    command_all_in_1 = command_download_frames +' && ' + command_get_1_frame +' && ' + command_del_video
    print(command_all_in_1)
    #1/0
    print('running donwload, extract, delete cmd...',end='')
    print(os.system(command_all_in_1))

    


    print('add file to sources')
    # add video id to sources
    line = str(filename+"\n")
    header_file = open("spamham_samples/header.txt","a")
    sources_file = open("spamham_samples/sources.txt","a")
    header_file.write(line)
    sources_file.write(video_channel+","+full_link+"\n")
    header_file.close()
    sources_file.close()
    
    print('label frame')
    # append (filename,1) to labels file# load labelled file in append mode
    with open('spamham_samples/manual_frames_labels.txt', 'a') as labels_file:
        line = str(filename+",1\n")
        labels_file.write(line)   

|UousROBIBT4|
get vid info
[youtube] Extracting URL: https://www.youtube.com/watch?v=UousROBIBT4
[youtube] UousROBIBT4: Downloading webpage
[youtube] UousROBIBT4: Downloading android player API JSON
[youtube] UousROBIBT4: Downloading MPD manifest
get random frame
*922-923
UousROBIBT4
yt-dlp -S res:720,ext:mp4,fps:24 -c --recode mp4 --download-sections *922-923 https://www.youtube.com/watch?v=UousROBIBT4 --output spamham_samples/UousROBIBT4.mp4 --retries infinite --fragment-retries infinite && ffmpeg -i spamham_samples/UousROBIBT4.mp4 -r 1 -vframes 1 -f image2 -q:v 1 spamham_samples/frames/UousROBIBT4.%03d.jpeg -y && del spamham_samples\UousROBIBT4.mp4
running donwload, extract, delete cmd...0
add file to sources
label frame
|UgEzkgti_eY|
get vid info
[youtube] Extracting URL: https://www.youtube.com/watch?v=UgEzkgti_eY
[youtube] UgEzkgti_eY: Downloading webpage
[youtube] UgEzkgti_eY: Downloading android player API JSON
[youtube] UgEzkgti_eY: Downloading MPD manifest
get random frame
*3

         Install PhantomJS to workaround the issue. Please download it from https://phantomjs.org/download.html
         n = f16kHG1UoVmfd9 ; player = https://www.youtube.com/s/player/97ea7458/player_ias.vflset/en_US/base.js


get random frame
*144-145
i-JUyqYPoSA
yt-dlp -S res:720,ext:mp4,fps:24 -c --recode mp4 --download-sections *144-145 https://www.youtube.com/watch?v=i-JUyqYPoSA --output spamham_samples/i-JUyqYPoSA.mp4 --retries infinite --fragment-retries infinite && ffmpeg -i spamham_samples/i-JUyqYPoSA.mp4 -r 1 -vframes 1 -f image2 -q:v 1 spamham_samples/frames/i-JUyqYPoSA.%03d.jpeg -y && del spamham_samples\i-JUyqYPoSA.mp4
running donwload, extract, delete cmd...0
add file to sources
label frame
|G3kCWi-eax0|
get vid info
[youtube] Extracting URL: https://www.youtube.com/watch?v=G3kCWi-eax0
[youtube] G3kCWi-eax0: Downloading webpage
[youtube] G3kCWi-eax0: Downloading android player API JSON
[youtube] G3kCWi-eax0: Downloading MPD manifest
get random frame
*61-62
G3kCWi-eax0
yt-dlp -S res:720,ext:mp4,fps:24 -c --recode mp4 --download-sections *61-62 https://www.youtube.com/watch?v=G3kCWi-eax0 --output spamham_samples/G3kCWi-eax0.mp4 --retries infinite --fragment-retries infinite && ffmpeg -i spamham_s

         Install PhantomJS to workaround the issue. Please download it from https://phantomjs.org/download.html
         n = gXuOfFwu8DgA89 ; player = https://www.youtube.com/s/player/97ea7458/player_ias.vflset/en_US/base.js


get random frame
*85-86
GfzbFKIy7D4
yt-dlp -S res:720,ext:mp4,fps:24 -c --recode mp4 --download-sections *85-86 https://www.youtube.com/watch?v=GfzbFKIy7D4 --output spamham_samples/GfzbFKIy7D4.mp4 --retries infinite --fragment-retries infinite && ffmpeg -i spamham_samples/GfzbFKIy7D4.mp4 -r 1 -vframes 1 -f image2 -q:v 1 spamham_samples/frames/GfzbFKIy7D4.%03d.jpeg -y && del spamham_samples\GfzbFKIy7D4.mp4
running donwload, extract, delete cmd...0
add file to sources
label frame
|zbvJlAKTiQo|
get vid info
[youtube] Extracting URL: https://www.youtube.com/watch?v=zbvJlAKTiQo
[youtube] zbvJlAKTiQo: Downloading webpage
[youtube] zbvJlAKTiQo: Downloading android player API JSON
[youtube] zbvJlAKTiQo: Downloading MPD manifest
get random frame
*704-705
zbvJlAKTiQo
yt-dlp -S res:720,ext:mp4,fps:24 -c --recode mp4 --download-sections *704-705 https://www.youtube.com/watch?v=zbvJlAKTiQo --output spamham_samples/zbvJlAKTiQo.mp4 --retries infinite --fragment-retries infinite && ffmpeg -i spamham_s

...

get random frame
*1485-1486
MTgFOPCzYB4
yt-dlp -S res:720,ext:mp4,fps:24 -c --recode mp4 --download-sections *1485-1486 https://www.youtube.com/watch?v=MTgFOPCzYB4 --output spamham_samples/MTgFOPCzYB4.mp4 --retries infinite --fragment-retries infinite && ffmpeg -i spamham_samples/MTgFOPCzYB4.mp4 -r 1 -vframes 1 -f image2 -q:v 1 spamham_samples/frames/MTgFOPCzYB4.%03d.jpeg -y && del spamham_samples\MTgFOPCzYB4.mp4
running donwload, extract, delete cmd...0
add file to sources
label frame
|CkjasKQN1Wc|
get vid info
[youtube] Extracting URL: https://www.youtube.com/watch?v=CkjasKQN1Wc
[youtube] CkjasKQN1Wc: Downloading webpage
[youtube] CkjasKQN1Wc: Downloading android player API JSON


         Install PhantomJS to workaround the issue. Please download it from https://phantomjs.org/download.html
         n = HknyIl8FWypHxO ; player = https://www.youtube.com/s/player/97ea7458/player_ias.vflset/en_US/base.js


get random frame
*1202-1203
CkjasKQN1Wc
yt-dlp -S res:720,ext:mp4,fps:24 -c --recode mp4 --download-sections *1202-1203 https://www.youtube.com/watch?v=CkjasKQN1Wc --output spamham_samples/CkjasKQN1Wc.mp4 --retries infinite --fragment-retries infinite && ffmpeg -i spamham_samples/CkjasKQN1Wc.mp4 -r 1 -vframes 1 -f image2 -q:v 1 spamham_samples/frames/CkjasKQN1Wc.%03d.jpeg -y && del spamham_samples\CkjasKQN1Wc.mp4
running donwload, extract, delete cmd...0
add file to sources
label frame
|-afmdsvCN2A|
get vid info
[youtube] Extracting URL: https://www.youtube.com/watch?v=-afmdsvCN2A
[youtube] -afmdsvCN2A: Downloading webpage
[youtube] -afmdsvCN2A: Downloading android player API JSON


         Install PhantomJS to workaround the issue. Please download it from https://phantomjs.org/download.html
         n = ZOWjqYl71SJpLB ; player = https://www.youtube.com/s/player/97ea7458/player_ias.vflset/en_US/base.js


get random frame
*201-202
-afmdsvCN2A
yt-dlp -S res:720,ext:mp4,fps:24 -c --recode mp4 --download-sections *201-202 https://www.youtube.com/watch?v=-afmdsvCN2A --output spamham_samples/-afmdsvCN2A.mp4 --retries infinite --fragment-retries infinite && ffmpeg -i spamham_samples/-afmdsvCN2A.mp4 -r 1 -vframes 1 -f image2 -q:v 1 spamham_samples/frames/-afmdsvCN2A.%03d.jpeg -y && del spamham_samples\-afmdsvCN2A.mp4
running donwload, extract, delete cmd...0
add file to sources
label frame
|APm14XlxN_c|
get vid info
[youtube] Extracting URL: https://www.youtube.com/watch?v=APm14XlxN_c
[youtube] APm14XlxN_c: Downloading webpage
[youtube] APm14XlxN_c: Downloading android player API JSON
[youtube] APm14XlxN_c: Downloading MPD manifest
get random frame
*3785-3786
APm14XlxN_c
yt-dlp -S res:720,ext:mp4,fps:24 -c --recode mp4 --download-sections *3785-3786 https://www.youtube.com/watch?v=APm14XlxN_c --output spamham_samples/APm14XlxN_c.mp4 --retries infinite --fragment-retries infinite && ffmpeg -i s